In [1]:
import polars as pl
import pandas as pd
from time import time
from pathlib import Path
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
round((Path("data.csv").stat().st_size / 1048576), 2)

566.11

In [4]:
def elapse_time(func):
    def wrapper(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        execution_time = round((end_time - start_time), 2)
        print(f"Function {func.__name__} took {execution_time} seconds to execute")
        return result
    return wrapper

In [5]:
@elapse_time
def read_csv(path: str, lib: str) -> pd.DataFrame | pl.DataFrame:
    if lib == "polars":
        df = pl.read_csv(path)
    elif lib == "pandas":
        df = pd.read_csv(path)
    else:
        raise TypeError("lib must be one of 'polars' or 'pandas'.")
    return df

In [6]:
pldf = read_csv("data.csv", "polars")

Function read_csv took 1.29 seconds to execute


In [23]:
pddf = read_csv("data.csv", "pandas")

Function read_csv took 11.23 seconds to execute


In [ ]:
# Schema

In [15]:
pldf.schema

Schema([('VendorID', Int64),
        ('tpep_pickup_datetime', String),
        ('tpep_dropoff_datetime', String),
        ('passenger_count', Int64),
        ('trip_distance', Float64),
        ('RatecodeID', Int64),
        ('store_and_fwd_flag', String),
        ('PULocationID', Int64),
        ('DOLocationID', Int64),
        ('payment_type', Int64),
        ('fare_amount', Float64),
        ('extra', Float64),
        ('mta_tax', Float64),
        ('tip_amount', Float64),
        ('tolls_amount', Float64),
        ('improvement_surcharge', Float64),
        ('total_amount', Float64),
        ('congestion_surcharge', Float64)])

In [14]:
pddf.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [ ]:
# Describe

In [19]:
pldf.describe()

statistic,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
str,f64,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",6.339567e6,"""6405008""","""6405008""",6.339567e6,6.405008e6,6.339567e6,"""6339567""",6.405008e6,6.405008e6,6.339567e6,6.405008e6,6.405008e6,6.405008e6,6.405008e6,6.405008e6,6.405008e6,6.405008e6,6.405008e6
"""null_count""",65441.0,"""0""","""0""",65441.0,0.0,65441.0,"""65441""",0.0,0.0,65441.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",1.669624,null,null,1.515333,2.929644,1.059908,null,164.732258,162.662691,1.270298,12.694108,1.115456,0.492318,2.189342,0.34884,0.297987,18.663149,2.275662
"""std""",0.470348,null,null,1.151594,83.159106,0.811843,null,65.543739,69.912606,0.473999,12.127295,1.260054,0.073742,2.760028,1.766978,0.033859,14.757364,0.735265
"""min""",1.0,"""2003-01-01 00:07:17""","""2003-01-01 14:16:59""",0.0,-30.62,1.0,"""N""",1.0,1.0,1.0,-1238.0,-27.0,-0.5,-91.0,-35.74,-0.3,-1242.3,-2.5
"""25%""",1.0,null,null,1.0,0.96,1.0,null,132.0,113.0,1.0,6.5,0.0,0.5,0.0,0.0,0.3,11.16,2.5
"""50%""",2.0,null,null,1.0,1.6,1.0,null,162.0,162.0,1.0,9.0,0.5,0.5,1.95,0.0,0.3,14.3,2.5
"""75%""",2.0,null,null,2.0,2.93,1.0,null,234.0,234.0,2.0,14.0,2.5,0.5,2.86,0.0,0.3,19.8,2.5
"""max""",2.0,"""2021-01-02 01:12:10""","""2021-01-02 01:25:01""",9.0,210240.07,99.0,"""Y""",265.0,265.0,5.0,4265.0,113.01,30.8,1100.0,910.5,0.3,4268.3,2.75


In [18]:
pddf.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,6.339567e+06,6.339567e+06,6.405008e+06,6.339567e+06,6.405008e+06,6.405008e+06,6.339567e+06,6.405008e+06,6.405008e+06,6.405008e+06,6.405008e+06,6.405008e+06,6.405008e+06,6.405008e+06,6.405008e+06
mean,1.669624e+00,1.515333e+00,2.929644e+00,1.059908e+00,1.647323e+02,1.626627e+02,1.270298e+00,1.269411e+01,1.115456e+00,4.923182e-01,2.189342e+00,3.488395e-01,2.979870e-01,1.866315e+01,2.275662e+00
std,4.703484e-01,1.151594e+00,8.315911e+01,8.118432e-01,6.554374e+01,6.991261e+01,4.739985e-01,1.212730e+01,1.260054e+00,7.374184e-02,2.760028e+00,1.766978e+00,3.385937e-02,1.475736e+01,7.352646e-01
min,1.000000e+00,0.000000e+00,-3.062000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.238000e+03,-2.700000e+01,-5.000000e-01,-9.100000e+01,-3.574000e+01,-3.000000e-01,-1.242300e+03,-2.500000e+00
25%,1.000000e+00,1.000000e+00,9.600000e-01,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.116000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.600000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.950000e+00,0.000000e+00,3.000000e-01,1.430000e+01,2.500000e+00
75%,2.000000e+00,2.000000e+00,2.930000e+00,1.000000e+00,2.340000e+02,2.340000e+02,2.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,2.860000e+00,0.000000e+00,3.000000e-01,1.980000e+01,2.500000e+00
max,2.000000e+00,9.000000e+00,2.102401e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,4.265000e+03,1.130100e+02,3.080000e+01,1.100000e+03,9.105000e+02,3.000000e-01,4.268300e+03,2.750000e+00


In [ ]:
# Add new column

In [32]:
pldf["new_column"] = "important info"

TypeError: DataFrame object does not support `Series` assignment by index

Use `DataFrame.with_columns`.

In [33]:
pldf = pldf.with_columns(
    new_column=pl.lit("important info")
)

In [34]:
pldf.head(3)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_distance_category,new_column
i64,str,str,i64,f64,i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
1,"""2020-01-01 00:28:15""","""2020-01-01 00:33:03""",1,1.2,1,"""N""",238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,"""1+""","""important info"""
1,"""2020-01-01 00:35:39""","""2020-01-01 00:43:04""",1,1.2,1,"""N""",239,238,1,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,"""1+""","""important info"""
1,"""2020-01-01 00:47:41""","""2020-01-01 00:53:52""",1,0.6,1,"""N""",238,238,1,6.0,3.0,0.5,1.0,0.0,0.3,10.8,2.5,"""0-1""","""important info"""


In [36]:
pldf = pldf.with_columns(
    pl.lit(1.3578490).alias("another_new_column")
)

In [37]:
pldf.head(3)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_distance_category,new_column,another_new_column
i64,str,str,i64,f64,i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64
1,"""2020-01-01 00:28:15""","""2020-01-01 00:33:03""",1,1.2,1,"""N""",238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,"""1+""","""important info""",1.657849
1,"""2020-01-01 00:35:39""","""2020-01-01 00:43:04""",1,1.2,1,"""N""",239,238,1,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,"""1+""","""important info""",1.657849
1,"""2020-01-01 00:47:41""","""2020-01-01 00:53:52""",1,0.6,1,"""N""",238,238,1,6.0,3.0,0.5,1.0,0.0,0.3,10.8,2.5,"""0-1""","""important info""",1.657849


In [38]:
pldf = pldf.with_columns(
    pl.col("another_new_column").cast(pl.Int64).alias("another_new_column_INT64")
)

In [39]:
pldf.head(3)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_distance_category,new_column,another_new_column,another_new_column_INT64
i64,str,str,i64,f64,i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,i64
1,"""2020-01-01 00:28:15""","""2020-01-01 00:33:03""",1,1.2,1,"""N""",238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,"""1+""","""important info""",1.657849,1
1,"""2020-01-01 00:35:39""","""2020-01-01 00:43:04""",1,1.2,1,"""N""",239,238,1,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,"""1+""","""important info""",1.657849,1
1,"""2020-01-01 00:47:41""","""2020-01-01 00:53:52""",1,0.6,1,"""N""",238,238,1,6.0,3.0,0.5,1.0,0.0,0.3,10.8,2.5,"""0-1""","""important info""",1.657849,1


In [ ]:
# Conditional formatting

In [31]:
pldf.head(3)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,trip_distance_category
i64,str,str,i64,f64,i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str
1,"""2020-01-01 00:28:15""","""2020-01-01 00:33:03""",1,1.2,1,"""N""",238,239,1,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,"""1+"""
1,"""2020-01-01 00:35:39""","""2020-01-01 00:43:04""",1,1.2,1,"""N""",239,238,1,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,"""1+"""
1,"""2020-01-01 00:47:41""","""2020-01-01 00:53:52""",1,0.6,1,"""N""",238,238,1,6.0,3.0,0.5,1.0,0.0,0.3,10.8,2.5,"""0-1"""


In [27]:
pldf = pldf.with_columns(
    pl.when(
        (pl.col("trip_distance")>0) & (pl.col("trip_distance")<1)
    )
    .then(pl.lit("0-1"))
    .otherwise(pl.lit("1+"))
    .alias("trip_distance_category")
)

In [28]:
pldf[["trip_distance", "trip_distance_category", "passenger_count"]].head(3)

trip_distance,trip_distance_category,passenger_count
f64,str,i64
1.2,"""1+""",1
1.2,"""1+""",1
0.6,"""0-1""",1


In [ ]:
# in Pandas

In [50]:
pddf.loc[(pddf['trip_distance']<1) & (pddf['trip_distance']>0), 'trip_distance_category'] = "0-1"
pddf.loc[pddf['trip_distance']>=1, 'trip_distance_category'] = "1+"

In [51]:
pddf[["trip_distance", "trip_distance_category", "passenger_count"]].head(3)

,trip_distance,trip_distance_category,passenger_count
0,1.2,1+,1.0
1,1.2,1+,1.0
2,0.6,0-1,1.0


In [ ]:
# Sum

In [42]:
pldf.select(pl.sum("trip_distance", "passenger_count"))

trip_distance,passenger_count
f64,i64
1.8764e7,9606553


In [ ]:
# Group By

In [27]:
pldf.group_by("trip_distance_category").agg(pl.col("passenger_count").sum())

trip_distance_category,passenger_count
str,i64
"""0-1""",2388778
"""1+""",7217775


In [45]:
pldf.group_by("trip_distance_category").agg(pl.col("trip_distance").mean())

trip_distance_category,trip_distance
str,f64
"""1+""",3.675475
"""0-1""",0.676678


In [ ]:
# Unique values

In [46]:
pldf["passenger_count"].unique()

passenger_count
i64
null
0
1
2
3
…
5
6
7


In [47]:
pldf["passenger_count"].n_unique()

11

In [ ]:
# User-defined Functions

In [40]:
def transform_column(value: str) -> str:
    return f"{value}_transformed"

out = pldf.select(pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8))

In [41]:
out.head(3)

trip_distance_category
str
"""1+_transformed"""
"""1+_transformed"""
"""0-1_transformed"""


In [ ]:
# Lazy and Eager execution

In [19]:
@elapse_time
def transform_polars_eager(path):
    df = pl.read_csv(path)
    df = df.with_columns(
        new_column=pl.lit("important info")
    )
    df = df.with_columns(
        pl.lit(1.3578490).alias("another_new_column")
    )
    df = df.with_columns(
        pl.col("another_new_column").cast(pl.Int64).alias("another_new_column_INT64")
    )
    df = df.with_columns(
        pl.when(
            (pl.col("trip_distance")>0) & (pl.col("trip_distance")<1)
        )
        .then(pl.lit("0-1"))
        .otherwise(pl.lit("1+"))
        .alias("trip_distance_category")
    )
    df = df.with_columns(
        pl.mean("trip_distance").alias("mean_trip_distance")
    )
    def transform_column(value: str) -> str:
        return f"{value}_transformed"
    
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_1")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_2")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_3")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_4")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_5")
    )

In [20]:
@elapse_time
def transform_polars_lazy(path):
    df = pl.scan_csv(path)
    df = df.with_columns(
        new_column=pl.lit("important info")
    )
    df = df.with_columns(
        pl.lit(1.3578490).alias("another_new_column")
    )
    df = df.with_columns(
        pl.col("another_new_column").cast(pl.Int64).alias("another_new_column_INT64")
    )
    df = df.with_columns(
        pl.when(
            (pl.col("trip_distance")>0) & (pl.col("trip_distance")<1)
        )
        .then(pl.lit("0-1"))
        .otherwise(pl.lit("1+"))
        .alias("trip_distance_category")
    )
    df = df.with_columns(
        pl.mean("trip_distance").alias("mean_trip_distance")
    )
    def transform_column(value: str) -> str:
        return f"{value}_transformed"
    
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_1")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_2")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_3")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_4")
    )
    df = df.with_columns(
        pl.col("trip_distance_category").map_elements(transform_column, return_dtype=pl.Utf8).alias("trip_distance_category_transformed_5")
    )

In [21]:
transform_polars_eager("data.csv")

Function transform_polars_eager took 9.04 seconds to execute


In [22]:
transform_polars_lazy("data.csv")

Function transform_polars_lazy took 0.0 seconds to execute


In [ ]:
@elapse_time
def transform_pandas(df: pd.DataFrame) -> pd.DataFrame:
    # TODO

In [23]:
# Visualization

In [13]:
pldf[:500].plot.bar(x="trip_distance", y="passenger_count")

alt.Chart(...)

In [15]:
pldf[:500].plot.point(x="trip_distance", y="tip_amount")

alt.Chart(...)